In [1]:
import pandas as pd

In [2]:
df_Top1000Delegates = pd.read_csv("./Top1000Delegates.csv")
df_vote_cast = pd.read_csv("./vote_cast.csv")
df_proposals = pd.read_csv("./proposals.csv")

In [3]:
df_proposals.head()

,Unnamed: 0,contract_address,evt_tx_hash,evt_tx_from,evt_tx_to,evt_index,evt_block_time,evt_block_number,evt_block_date,calldatas,...,endBlock,proposalData,proposalId,proposalType,proposer,signatures,startBlock,targets,values,votingModule
0,0,0xcdf27f107725988f2261ce2256bdfcde8b382b10,0x30fb20241ffe79b6765def34f4307993efe33afc7c4e...,0x47c88bb92b409ff25f6587ea611fac4e55f76007,0xe4553b743e74da3424ac51f8c1e586fd43ae226f,190,2024-02-29 19:39:35.000 UTC,116818399.0,2024-02-29,[0x],...,117077599,<nil>,6209220216819577389492395498819206401301553603...,0,0xe4553b743e74da3424ac51f8c1e586fd43ae226f,[],116818399,[0x0000000000000000000000000000000000000000],[0],<nil>
1,1,0xcdf27f107725988f2261ce2256bdfcde8b382b10,0x70b9f4dd27d551c12f0169e87626c3564d46c1318210...,0x47c88bb92b409ff25f6587ea611fac4e55f76007,0xe4553b743e74da3424ac51f8c1e586fd43ae226f,272,2024-02-29 19:47:57.000 UTC,116818650.0,2024-02-29,[0x],...,117077850,<nil>,2782124307695730501270771393458700341474277310...,0,0xe4553b743e74da3424ac51f8c1e586fd43ae226f,[],116818650,[0x0000000000000000000000000000000000000000],[0],<nil>
2,2,0xcdf27f107725988f2261ce2256bdfcde8b382b10,0xf862cc30322e61461bbfa5eb0e9f43e967eb98628378...,0x47c88bb92b409ff25f6587ea611fac4e55f76007,0xa1dabef33b3b82c7814b6d82a79e50f4ac44102b,40,2024-02-29 21:51:37.000 UTC,116822360.0,2024-02-29,[0x],...,117081560,<nil>,4725311336691981283179142257151334707337482850...,1,0xe4553b743e74da3424ac51f8c1e586fd43ae226f,[],116822360,[0x0000000000000000000000000000000000000000],[0],<nil>
3,3,0xcdf27f107725988f2261ce2256bdfcde8b382b10,0xf862cc30322e61461bbfa5eb0e9f43e967eb98628378...,0x47c88bb92b409ff25f6587ea611fac4e55f76007,0xa1dabef33b3b82c7814b6d82a79e50f4ac44102b,37,2024-02-29 21:51:37.000 UTC,116822360.0,2024-02-29,[0x],...,117081560,<nil>,9511969859771175018673437798469781410170719088...,1,0xe4553b743e74da3424ac51f8c1e586fd43ae226f,[],116822360,[0x0000000000000000000000000000000000000000],[0],<nil>
4,4,0xcdf27f107725988f2261ce2256bdfcde8b382b10,0xab85790d60ee7aadcddb02842bbb50d4fa9535b2ae83...,0x84fb5b7a35cb08a5c8a14d8aac537436027d6c5b,0xe4553b743e74da3424ac51f8c1e586fd43ae226f,52,2024-04-11 18:53:31.000 UTC,118631417.0,2024-04-11,[0x],...,118890617,<nil>,4343198946866183739413628804128419161742568907...,1,0xe4553b743e74da3424ac51f8c1e586fd43ae226f,[],118631417,[0x0000000000000000000000000000000000000000],[0],<nil>
